In [1]:
from google.colab import drive
import os
import pandas as pd
from tqdm import tqdm
drive.mount('/content/gdrive', force_remount=True)

root_dir = "/content/gdrive/MyDrive/"
base_dir = root_dir + 'JSONS/'
os.listdir(base_dir)

#Load malicious ip dataframe
malicious_ips = pd.read_json(base_dir+"malicious_ips.json")
malicious_ips.rename(columns = {0 : "id.orig_h"}, inplace = True)

malicious_ips

MessageError: ignored

In [ ]:

def read_and_convert_timestamp(file_path):
    # Read JSON file into a DataFrame
    frame = pd.read_json(base_dir + file_path)

    # Convert the 'ts' column to datetime format
    frame['ts'] = pd.to_datetime(frame['ts'], format='%Y-%m-%dT%H:%M:%S.%f')

    # Return the DataFrame with updated timestamp
    return frame

def filter_and_reset_index(frame, port, ip_filter, malicious_ips):
    # Filter out rows with 'id.orig_h' not in the list of malicious IPs
    filtered_good = frame[~frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)

    # Filter rows with 'id.orig_h' in the list of malicious IPs
    filtered_malicious = frame[frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)

    # Return the filtered DataFrames for good and malicious traffic
    return filtered_good, filtered_malicious


In [ ]:
# Read JSON file into a DataFrame
http_frame = pd.read_json(base_dir + "http.json")

# Convert 'ts' column to datetime format
http_frame['ts'] = pd.to_datetime(http_frame['ts'], format='%Y-%m-%dT%H:%M:%S.%f')

# Filter HMI and PLC HTTP frames
hmi_http = http_frame[((http_frame["id.resp_p"].isin([8081, 8082])) & (http_frame["id.resp_h"] == "192.168.10.114"))]
plc_http = http_frame[(http_frame["id.resp_p"] == 8080)]

# Filter HMI HTTP frames for good and malicious IPs
hmi_http_good = hmi_http[~hmi_http['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)
hmi_http_malicious = hmi_http[hmi_http['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)

# Filter PLC HTTP frames for good and malicious IPs
plc_http_good = plc_http[~plc_http['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)
plc_http_malicious = plc_http[plc_http['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)


# Read Modbus JSON file into a DataFrame
modbus_frame = pd.read_json(base_dir + "modbus.json")

# Convert 'ts' column to datetime format if needed
modbus_frame['ts'] = pd.to_datetime(modbus_frame['ts'], format='%Y-%m-%dT%H:%M:%S.%f')

# Apply filters and reset index for good and malicious Modbus frames
modbus_good = modbus_frame[~modbus_frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)
modbus_malicious = modbus_frame[modbus_frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)

# Read S7Comm JSON file into a DataFrame
s7comm_frame = pd.read_json(base_dir + "s7comm.json")

# Convert 'ts' column to datetime format if needed
s7comm_frame['ts'] = pd.to_datetime(s7comm_frame['ts'], format='%Y-%m-%dT%H:%M:%S.%f')
s7comm_frame = s7comm_frame[(s7comm_frame["ts"] > pd.to_datetime("2023-06-03T00:42:50.572273920", format='%Y-%m-%dT%H:%M:%S.%f'))]
# Apply filters and reset index for good and malicious S7Comm frames
s7comm_good = s7comm_frame[~s7comm_frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)
s7comm_malicious = s7comm_frame[s7comm_frame['id.orig_h'].isin(malicious_ips['id.orig_h'])].reset_index(drop=True)

In [ ]:
# Print length of HTTP frames
print("Length of HMI HTTP frames:", len(hmi_http))
print("Length of PLC HTTP frames:", len(plc_http))
print("Length of HMI HTTP good frames:", len(hmi_http_good))
print("Length of HMI HTTP malicious frames:", len(hmi_http_malicious))
print("Length of PLC HTTP good frames:", len(plc_http_good))
print("Length of PLC HTTP malicious frames:", len(plc_http_malicious))

In [ ]:
# Print length of Modbus frames
print("Length of Modbus frames:", len(modbus_frame))
print("Length of Modbus good frames:", len(modbus_good))
print("Length of Modbus malicious frames:", len(modbus_malicious))

In [ ]:

# Print length of S7Comm frames
print("Length of S7Comm frames:", len(s7comm_frame))
print("Length of S7Comm good frames:", len(s7comm_good))
print("Length of S7Comm malicious frames:", len(s7comm_malicious))

In [ ]:
labels = [
     'HTTP HMI (benign)',
    'HTTP PLC (benign)',
    'MODBUS (benign)',
    'S7Comm (benign)',
    'HTTP HMI (malicious)',
    'HTTP PLC (malicious)',
    'MODBUS (malicious)',
    'S7Comm (malicious)'
]

#labels = ['IT (benign)', 'ICS (benign)', 'IT (malicious)', 'ICS (malicious)']
#Specify the session time limit in seconds for each protocol/container type here.
bounds = dict()
bounds['HTTP HMI (benign)'] = 30*60
bounds['HTTP PLC (benign)'] = 5
bounds['MODBUS (benign)'] = 1
bounds['S7Comm (benign)'] = 1

bounds['HTTP HMI (malicious)'] = 30*60
bounds['HTTP PLC (malicious)'] = 5
bounds['MODBUS (malicious)'] = 1
bounds['S7Comm (malicious)'] = 1

In [ ]:
def split_sessions_start_time(label, table):
    time_delta = pd.Timedelta(seconds=bounds[label])

    # Sort the DataFrame
    table.sort_values(['id.orig_h', 'id.resp_h', 'id.orig_p', 'id.resp_p', 'ts'], inplace=True)

    sessions = []

    # Group by source ip, port and destination ip, port
    for _, group in tqdm(table.groupby(['id.orig_h', 'id.resp_h', 'id.orig_p', 'id.resp_p'])):
        if group.empty:
            continue

        session = []
        #Take time of previous request
        last_time = group.iloc[0]['ts']

        for _, row in group.iterrows():
          #If the timestamp of the current request exceeds the sum of the previous timestamp and the specified time threshold, add the session to the list of sessions.
            if row['ts'] > last_time + time_delta:
                sessions.append(pd.DataFrame(session))
                session = []

            last_time = row['ts']
            session.append(row)

        if session:
            sessions.append(pd.DataFrame(session))

    return sessions

In [ ]:
interaction_list = [(labels[0] , split_sessions_start_time(labels[0], hmi_http_good)),
                    (labels[1] , split_sessions_start_time(labels[1], plc_http_good)),
                    (labels[2], split_sessions_start_time(labels[2], modbus_good)),
                    (labels[3], split_sessions_start_time(labels[3], s7comm_good)),
                    (labels[4] , split_sessions_start_time(labels[4], hmi_http_malicious)),
                    (labels[5] , split_sessions_start_time(labels[5], plc_http_malicious)),
                    (labels[6], split_sessions_start_time(labels[6], modbus_malicious)),
                    (labels[7], split_sessions_start_time(labels[7], s7comm_malicious))]

6.2.1 RQ5: From which geographic regions do the observed ICS
interactions originate compared to IT interactions?

In [ ]:
for i in range(len(interaction_list)):
    print(interaction_list[i][0], len(interaction_list[i][1]))

In [ ]:
def count_interactions(dfx):
  mappa_ip_count = dict()
  for i in range(len(dfx)):
      x = dfx[i].groupby(['id.orig_h']).count()['ts'].reset_index()
      x_ip_seen = set()
      for ind in x.index:
          ip = x['id.orig_h'][ind]
          if ip not in x_ip_seen:
              if ip in mappa_ip_count:
                  mappa_ip_count[ip] += 1
              else:
                  mappa_ip_count[ip] = 1
              x_ip_seen.add(ip)

  return mappa_ip_count

In [ ]:
get_sess_lengths = lambda lst: pd.Series([x.shape[0] for x in lst])

In [ ]:
table = []
ind = []
for label, values in interaction_list:
    tbl_data = get_sess_lengths(values)
    table.append([tbl_data.min(),
                  tbl_data.max(),
                  tbl_data.median(),
                  tbl_data.quantile(0.25),
                  tbl_data.quantile(0.75),
                  tbl_data.shape[0]])
    ind.append(label)
pd.DataFrame(table,
             index=ind,
             columns = ['min', 'max', 'median', 'Q1', 'Q3', 'count'])

In [ ]:
hmi_http_good = pd.DataFrame(count_interactions(interaction_list[0][1]).items(), columns=['ip', 'interactions'])
plc_http_good = pd.DataFrame(count_interactions(interaction_list[1][1]).items(), columns=['ip', 'interactions'])

hmi_http_malicious = pd.DataFrame(count_interactions(interaction_list[4][1]).items(), columns=['ip', 'interactions'])
plc_http_malicious = pd.DataFrame(count_interactions(interaction_list[5][1]).items(), columns=['ip', 'interactions'])

IT_good = pd.concat([hmi_http_good,plc_http_good])
IT_malicious = pd.concat([hmi_http_malicious,plc_http_malicious])

modbus_good = pd.DataFrame(count_interactions(interaction_list[2][1]).items(), columns=['ip', 'interactions'])
s7comm_good = pd.DataFrame(count_interactions(interaction_list[3][1]).items(), columns=['ip', 'interactions'])
modbus_malicious = pd.DataFrame(count_interactions(interaction_list[6][1]).items(), columns=['ip', 'interactions'])
s7comm_malicious = pd.DataFrame(count_interactions(interaction_list[7][1]).items(), columns=['ip', 'interactions'])

ICS_good = pd.concat([modbus_good, s7comm_good])
ICS_malicious = pd.concat([modbus_malicious, s7comm_malicious])

In [ ]:
import requests




ics_good = ICS_good
ics_bad = ICS_malicious
ics = pd.concat([ics_good,ics_bad])
ics.rename(columns={'id.orig_h':'ip'}, inplace=True)
ics.rename(columns={'count':'requests'}, inplace=True)

it_good = IT_good
it_bad = IT_malicious
it = pd.concat([it_good,it_bad])
it.rename(columns={'id.orig_h':'ip'}, inplace=True)
it.rename(columns={'count':'requests'}, inplace=True)

#i need to drop the request column and replace it with the interaction column coming from the IPvsInteractions.csv
malicious_ips = pd.read_json(base_dir+"malicious_ips.json")

ipvsinteraction = pd.read_csv(base_dir+"IPvsInteractions.csv")
ipvsinteraction.rename(columns={'count':'interactions'}, inplace=True)
ipvsinteraction.rename(columns={'id.orig_h':'ip'}, inplace=True)

ics = pd.merge(ics, ipvsinteraction, on='ip', how='left')
it = pd.merge(it, ipvsinteraction, on='ip', how='left')

#drop the requests column

#rename the interactions column in requests
ics.rename(columns={'interactions':'requests'}, inplace=True)
it.rename(columns={'interactions':'requests'}, inplace=True)

geo = pd.read_csv(base_dir+"merged.csv")
#drop all the columns but ip, classification, country, country_code, organization, actor and organization
geo = geo[['ip','classification','country','country_code','organization','actor']]
nan = geo[geo.isna().any(axis=1)]

#how many ips in ics are not present in geo
ics_not_in_geo = ics[~ics.ip.isin(geo.ip)]
#how many ips in it are not present in geo
it_not_in_geo = it[~it.ip.isin(geo.ip)]

#I need to merge the geo info with the ics and the geo info
#The ip which are not present in geo will appear in the new dataframe as NaN
ics_geo = pd.merge(ics, geo, on='ip', how='left')

#get the ips which are not present in geo, where just country is NaN
# assuming the dataframe is called df and the column you want to check is called 'column_name'
ics_nan = ics_geo.loc[ics_geo['country'].isnull()]
it_geo = pd.merge(it, geo, on='ip', how='left')
it_nan = it_geo.loc[it_geo['country'].isnull()]


error = 0
for ip in ics_nan.ip:
    respone = ""
    try:
        respone = requests.get(f'https://ipinfo.io/{ip}?token=#############')
        print(f'https://ipinfo.io/{ip}?token=################')
        print(respone.json())
    except:
        print(respone)
        print(ip)
        error += 1
    try:
        ics_geo.loc[ics_geo['ip'] == ip, 'country'] = respone.json()['region']
    except:
        ics_geo.loc[ics_geo['ip'] == ip, 'country'] = 'Unknown'
        error += 1
    try:
        ics_geo.loc[ics_geo['ip'] == ip, 'country_code'] = respone.json()['country']
    except:
        ics_geo.loc[ics_geo['ip'] == ip, 'country_code'] = 'Unknown'
        error += 1
    try:
        ics_geo.loc[ics_geo['ip'] == ip, 'organization'] = respone.json()['org']
        error += 1
    except:
        ics_geo.loc[ics_geo['ip'] == ip, 'organization'] = 'Unknown'
        error += 1
    ics_geo.loc[ics_geo['ip'] == ip, 'actor'] = 'Unknown'
    ics_geo.loc[ics_geo['ip'] == ip, 'classification'] = 'Unknown'

print('ICSerror: ',error)

error = 0
for ip in it_nan.ip:
    try:
        respone = requests.get(f'https://ipinfo.io/{ip}?token=#############')
    except:
        print('EEError')
        print(ip)
        print(respone)
        error += 1
    try:
        it_geo.loc[it_geo['ip'] == ip, 'country'] = respone.json()['region']
    except:
        it_geo.loc[it_geo['ip'] == ip, 'country'] = 'Unknown'
        error += 1
    try:
        it_geo.loc[it_geo['ip'] == ip, 'country_code'] = respone.json()['country']
    except:
        it_geo.loc[it_geo['ip'] == ip, 'country_code'] = 'Unknown'
        error += 1
    try:
        it_geo.loc[it_geo['ip'] == ip, 'organization'] = respone.json()['org']
    except:
        it_geo.loc[it_geo['ip'] == ip, 'organization'] = 'Unknown'
        error += 1
    it_geo.loc[it_geo['ip'] == ip, 'actor'] = 'Unknown'
    it_geo.loc[it_geo['ip'] == ip, 'classification'] = 'Unknown'

print('ITerror: ',error)








In [ ]:
it = it_geo
ics = ics_geo



#group by country print the number or IP and the sum of requests sorted by requests
it_grouped_sortedReq = it.groupby(['country']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['interactions_x'],ascending=False)
ics_grouped_sortedReq = ics.groupby(['country']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['interactions_x'],ascending=False)

#group by country print the number or IP and the sum of requests sorted by IP
it_grouped_sortedIP = it.groupby(['country']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['ip'],ascending=False)
ics_grouped_sortedIP = ics.groupby(['country']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['ip'],ascending=False)

print(it_grouped_sortedReq)
print(ics_grouped_sortedReq)
print(it_grouped_sortedIP)
print(ics_grouped_sortedIP)


RQ6: From which actors do the observed ICS interactions originate compared to IT interactions?

In [ ]:
it = it_geo
ics = ics_geo

#for each row if actor = unknown then copy the value from organization to actor
for index, row in it.iterrows():
    if row['actor'] == 'Unknown' or row['actor'] == 'unknown':
        it.loc[index,'actor'] = row['organization']

for index, row in ics.iterrows():
    if row['actor'] == 'Unknown' or row['actor'] == 'unknown':
        ics.loc[index,'actor'] = row['organization']

#remove " from actor column
it['actor'] = it['actor'].str.replace('"','')
ics['actor'] = ics['actor'].str.replace('"','')
it['organization'] = it['organization'].str.replace('"','')
ics['organization'] = ics['organization'].str.replace('"','')

#group by actor print the number or IP and the sum of requests sorted by requests
it_grouped_sortedReq = it.groupby(['actor']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['interactions_x'],ascending=False)
ics_grouped_sortedReq = ics.groupby(['actor']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['interactions_x'],ascending=False)

#group by actor print the number or IP and the sum of requests sorted by IP
it_grouped_sortedIP = it.groupby(['actor']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['ip'],ascending=False)
ics_grouped_sortedIP = ics.groupby(['actor']).agg({'ip':'count','interactions_x':'sum'}).sort_values(by=['ip'],ascending=False)

print(it_grouped_sortedReq)
print(ics_grouped_sortedReq)
print(it_grouped_sortedIP)
print(ics_grouped_sortedIP)



RQ7

In [ ]:
it = it_geo
ics = ics_geo

#see which ips are in both datasets, and create  a new dataframe with only those ips and the respective country
it_ics = pd.merge(it, ics, on='ip', how='inner')
it_ics = it_ics[['ip', 'country_x']]
it_ics = it_ics.rename(columns={'country_x': 'country'})
it_ics = it_ics.drop_duplicates(subset=['ip'])

print(it_ics)

ipvsinteraction = pd.read_csv(base_dir+"IPvsInteractions.csv")
ipvsinteraction.rename(columns={'count':'interactions'}, inplace=True)
ipvsinteraction.rename(columns={'id.orig_h':'ip'}, inplace=True)

#merge the new dataframe with the interactions
it_ics = pd.merge(it_ics, ipvsinteraction, on='ip', how='left')
#sort by interactions
it_ics = it_ics.sort_values(by=['interactions'],ascending=False)
print(it_ics)

#group by country and sum the interactions
it_ics_grouped = it_ics.groupby(['country']).agg({'interactions':'sum'}).sort_values(by=['interactions'],ascending=False)
print(it_ics_grouped)